dodatno: 

https://github.com/sdimi/average-word2vec/blob/master/notebook.ipynb

https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

https://stackoverflow.com/questions/18424228/cosine-similarity-between-2-number-lists

In [102]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib
import enchant
import re
import math
from collections import Counter
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import gensim
import string
import fasttext
import fasttext.util
from scipy import spatial
from gensim.parsing.preprocessing import remove_stopwords
import gensim.downloader as api

In [29]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

# Load model and csv

In [30]:
goog_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [31]:
df = pd.read_csv('datoteka_za_rad.csv')

# Calculate average vector for all words

In [32]:
def avg_feature_vector(sentence, model, num_features, index2word_set):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

# Functions

In [38]:
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [39]:
a_string = '!hi. wh?at is the weat[h]er lik?e.'

new_string = a_string.translate(str.maketrans('', '', string.punctuation))
print(new_string)

hi what is the weather like


In [40]:
df.reset_index()

for index, row in df.iterrows():
    for sentence in split_into_sentences(row['articleBody']):
        print(remove_stopwords(sentence.lower()))
        print('\n')
        print(sentence)
        break
    break

small meteorite crashed wooded area nicaragua's capital managua overnight, government said sunday.


A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.


In [72]:
def similarity_cosine(vec1, vec2):
    result = 1 - spatial.distance.cosine(vec1, vec2)
    return result

In [73]:
df.reset_index()

for index, row in df.iterrows():
    for sentence in split_into_sentences(row['articleBody']):
        #remove all
        sentence = remove_stopwords(sentence.translate(str.maketrans('', '', string.punctuation)).lower())
        
        #print
        print(sentence)
        break
    break

small meteorite crashed wooded area nicaraguas capital managua overnight government said sunday


In [74]:
def preprocess(text):
    return remove_stopwords(text.translate(str.maketrans('', '', string.punctuation)).lower())

In [75]:
def document_vector(word2vec_model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.key_to_index]
    return np.mean(word2vec_model[doc], axis=0)

# Average word2vec cosine similarity

In [96]:
w2v_results = pd.DataFrame(columns = ['w2v', 'Stance'])

In [76]:
df.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


In [97]:
df.reset_index()

j=0
for index, row in df.iterrows():
    
    #remove all
    article = preprocess(row['articleBody'])
    title = preprocess(row['Headline'])
    
    #tokenize
    title_tokens = word_tokenize(title)
    article_tokens = word_tokenize(article)
    
    #document vector
    title_vector = document_vector(goog_model, title_tokens)
    article_vector = document_vector(goog_model, article_tokens)
    
    similarity = similarity_cosine(title_vector, article_vector)
    
    #append
    w2v_results.loc[j] = [similarity, row['Stance']]
    j += 1

In [98]:
w2v_results

,w2v,Stance
0,0.490024,unrelated
1,0.474561,unrelated
2,0.430289,unrelated
3,0.438596,unrelated
4,0.498350,unrelated
...,...,...
49967,0.603139,related
49968,0.628640,related
49969,0.653130,related
49970,0.719600,related


In [99]:
w2v_results_related = w2v_results[w2v_results.Stance == 'related']
w2v_results_unrelated = w2v_results[w2v_results.Stance == 'unrelated']

In [100]:
w2v_results_related['w2v'].quantile([.05, .1, .15, .2, .25, .3, .35, .4, .45, .5, .55, .6, .65, .7, .75, .8, .85, .9, .95])

0.05    0.505183
0.10    0.557381
0.15    0.586389
0.20    0.609043
0.25    0.625930
0.30    0.640007
0.35    0.651949
0.40    0.664216
0.45    0.675153
0.50    0.685837
0.55    0.695183
0.60    0.705208
0.65    0.715308
0.70    0.725119
0.75    0.735672
0.80    0.746961
0.85    0.761105
0.90    0.777249
0.95    0.800237
Name: w2v, dtype: float64

In [101]:
w2v_results_unrelated['w2v'].quantile([.05, .1, .15, .2, .25, .3, .35, .4, .45, .5, .55, .6, .65, .7, .75, .8, .85, .9, .95])

0.05    0.302036
0.10    0.336155
0.15    0.359619
0.20    0.378138
0.25    0.394565
0.30    0.408731
0.35    0.422623
0.40    0.435559
0.45    0.448053
0.50    0.460214
0.55    0.472428
0.60    0.485103
0.65    0.497238
0.70    0.510341
0.75    0.524514
0.80    0.539441
0.85    0.557597
0.90    0.580397
0.95    0.613290
Name: w2v, dtype: float64